In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import T5Config, TFT5Model, T5Model, load_tf_weights_in_t5, T5Tokenizer
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration

2022-07-11 18:59:16.217758: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-11 18:59:16.217778: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-07-11 18:59:16.217782: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-07-11 18:59:16.217817: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-07-11 18:59:16.217836: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-07-11 18:59:16.218135: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F 

In [3]:
!rm -rf temp

In [4]:
import os

out = 'temp'
os.makedirs(out, exist_ok=True)

In [5]:
config = T5Config.from_pretrained('malay-huggingface/t5-small-bahasa-cased')
config.save_pretrained(out)

In [6]:
!ls t5-small-noisy-ms-en

checkpoint
events.out.tfevents.1657262576.huseincomel-desktop
events.out.tfevents.1657273096.huseincomel-desktop
events.out.tfevents.1657378269.huseincomel-desktop
events.out.tfevents.1657386478.huseincomel-desktop
events.out.tfevents.1657527614.huseincomel-desktop
graph.pbtxt
model.ckpt-1023200.data-00000-of-00002
model.ckpt-1023200.data-00001-of-00002
model.ckpt-1023200.index
model.ckpt-1023200.meta
model.ckpt-1048200.data-00000-of-00002
model.ckpt-1048200.data-00001-of-00002
model.ckpt-1048200.index
model.ckpt-1048200.meta
model.ckpt-1074200.data-00000-of-00002
model.ckpt-1074200.data-00001-of-00002
model.ckpt-1074200.index
model.ckpt-1074200.meta
model.ckpt-1099200.data-00000-of-00002
model.ckpt-1099200.data-00001-of-00002
model.ckpt-1099200.index
model.ckpt-1099200.meta
model.ckpt-1125200.data-00000-of-00002
model.ckpt-1125200.data-00001-of-00002
model.ckpt-1125200.index
model.ckpt-1125200.meta
model.ckpt-1150200.data-00000-of-00002
model.ckpt-1150200.data-00001-of-00002
model.ckp

In [7]:
model = T5Model(config)
load_tf_weights_in_t5(model, config, 't5-small-noisy-ms-en/model.ckpt-1573200')

T5Model(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): FusedRMSNorm(torch.Size([512]), eps=1e-06, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=

In [8]:
tokenizer = T5Tokenizer('sp10m.cased.ms-en.model', padding = True)

In [9]:
model.save_pretrained(out)

In [10]:
model_gen = T5ForConditionalGeneration.from_pretrained(out)

In [11]:
model_tf = TFT5ForConditionalGeneration.from_pretrained(out, from_pt = True)

2022-07-11 18:59:49.091230: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained 

In [12]:
with open('ms-en-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('ms-en-left.test') as fopen:
    left = fopen.read().split('\n')

In [13]:
# model_gen.cuda()

In [14]:
# from tqdm import tqdm

# batch_size = 16

# results = []
# for i in tqdm(range(0, len(left), batch_size)):
#     input_ids = [{'input_ids': tokenizer.encode(f'terjemah Melayu ke Inggeris: {s}', return_tensors = 'pt')[0]} for s in left[i:i + batch_size]]
#     padded = tokenizer.pad(input_ids, padding = 'longest')
#     outputs = model_gen.generate(padded['input_ids'].cuda(), attention_mask = padded['attention_mask'].cuda(), max_length = 1000)
#     for o in outputs:
#         results.append(tokenizer.decode(o, skip_special_tokens=True))

In [15]:
from tqdm import tqdm

batch_size = 16

results = []
for i in tqdm(range(0, len(left[:30]), batch_size)):
    input_ids = [{'input_ids': tokenizer.encode(f'terjemah Melayu ke Inggeris: {s}', return_tensors = 'pt')[0]} for s in left[i:i + batch_size]]
    padded = tokenizer.pad(input_ids, padding = 'longest')
    outputs = model_gen.generate(**padded, max_length = 1000)
    for o in outputs:
        results.append(tokenizer.decode(o, skip_special_tokens=True))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.07s/it]


In [16]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()

In [17]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [18]:
refs = [filtered_right]
sys = filtered_left

In [19]:
for i in range(len(filtered_left)):
    print(filtered_left[i])
    print(filtered_right[i])
    print(left[i])
    print()

The Penal Code (Malaysia) makes any sexual relationship between men and women under the age of 16 a rape whether or not it is committed by the woman's will.
The Penal Code (Malaysia) makes any sexual contact between a man and a woman under the age of 16 rape whether it is done at the will of the woman.
Kanun Keseksaan (Malaysia) menjadikan sebarang hubungan seksual antara lelaki dengan wanita di bawah umur 16 tahun sebagai rogol sama ada dilakukan dengan kerelaan wanita itu atau tidak.

The "waling-waling" geese, also known as the "Queen of the Philippines" and also one of the national symbols of Malaysia, are also endemic in the region.
The "orchid" orchid, also known as the "Queen of the Philippines" and also a national symbol of Malaysia, is also endemic in the region.
Anggrek "waling-waling", yang juga dikenali sebagai "Ratu Bunga Filipina" dan juga salah satu simbol kebangsaan negara Malaysia, juga endemik dalam kawasan.

God is All-knowing and All-wise.
Allah is All-Knowing, All-

In [20]:
filtered_right

['The Penal Code (Malaysia) makes any sexual contact between a man and a woman under the age of 16 rape whether it is done at the will of the woman.',
 'The "orchid" orchid, also known as the "Queen of the Philippines" and also a national symbol of Malaysia, is also endemic in the region.',
 'Allah is All-Knowing, All-Wise.',
 'Neighborhood, City of Buildings, City of Kartanegara Neighborhood is a village in the village of Bangun, Regency of Kutai Kartanegara, East Kalimantan, Indonesia.',
 'According to the Indian census in 2001 Allapuram had an education rate (reading ability) of 76%, above the national average of 59.5%; with 53% of men and 47% of women able to read.',
 'He was sponsored by Li-Ning.',
 'The Federal Audit Report 2002 states that the project was approved privatization through a land acquisition by the government in 1996 and began physical work by a consortium of two companies in May 2000.',
 'Tawau MRSM was initially expected to start operations in February 2008 but be

In [21]:
filtered_left

["The Penal Code (Malaysia) makes any sexual relationship between men and women under the age of 16 a rape whether or not it is committed to the woman's will.",
 'The "waling-waling" quake, also known as the "Queen of the Philippines" and also one of the national symbols of Malaysia, is also endemic in the area.',
 'God is All-knowing, All-wise.',
 'Sebelimbingan, Kota Bangun, Kutai Kartanegara Sebelimbingan is a village in the district of Kota Bangun, Regency of Kutai Kartanegara, East Kalimantan, Indonesia.',
 'According to the Indian census in 2001 Allapuram had an education rate (reading ability) of 76%, above the national average of 59.5%; with 53% of men and 47% of women able to read.',
 'He was sponsored by Li-Ning.',
 'The 2002 Federal Audit Report states that the project was approved by privatization through land exchange by the government in 1996 that began its physical work by a two-corporate consortium in May 2000.',
 'Tawau MRSM was initially expected to start operating in

In [20]:
r = bleu.corpus_score(sys, refs)

In [21]:
r.__dict__

{'name': 'BLEU',
 'score': 65.52581746285784,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '85.3/70.0/60.8/53.2 (BP = 0.989 ratio = 0.989 hyp_len = 701 ref_len = 709)',
 'bp': 0.9886526047274109,
 'counts': [598, 468, 387, 322],
 'totals': [701, 669, 637, 605],
 'sys_len': 701,
 'ref_len': 709,
 'precisions': [85.30670470756063,
  69.95515695067265,
  60.75353218210361,
  53.22314049586777],
 'prec_str': '85.3/70.0/60.8/53.2',
 'ratio': 0.9887165021156559}

In [22]:
tokenizer.push_to_hub('t5-small-finetuned-noisy-ms-en', organization='mesolitica')

In [23]:
model_gen.push_to_hub('t5-small-finetuned-noisy-ms-en', organization='mesolitica')

Upload file pytorch_model.bin:   0%|          | 4.00k/231M [00:00<?, ?B/s]

To https://huggingface.co/mesolitica/t5-small-finetuned-noisy-ms-en
   b9391eb..df949d6  main -> main



'https://huggingface.co/mesolitica/t5-small-finetuned-noisy-ms-en/commit/df949d638df32299f4ba271fabe0352b187e2fcf'

In [24]:
model_tf.push_to_hub('t5-small-finetuned-noisy-ms-en', organization='mesolitica')

Upload file tf_model.h5:   0%|          | 4.00k/231M [00:00<?, ?B/s]

To https://huggingface.co/mesolitica/t5-small-finetuned-noisy-ms-en
   df949d6..cd4e0aa  main -> main



'https://huggingface.co/mesolitica/t5-small-finetuned-noisy-ms-en/commit/cd4e0aa002a21a737839abed3887a567203b687e'

In [25]:
!cd t5-small-finetuned-noisy-ms-en && git pull

Already up to date.


In [26]:
!cp t5-small-noisy-ms-en/*.tfevents.* t5-small-finetuned-noisy-ms-en
!cd t5-small-finetuned-noisy-ms-en && git add . && git commit -m 'add tensorboard' && git push

[main d0a4b4b] add tensorboard
 2 files changed, 5 insertions(+), 2 deletions(-)
 create mode 100644 events.out.tfevents.1657527614.huseincomel-desktop
Uploading LFS objects: 100% (2/2), 114 MB | 4.3 MB/s, done.                     
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 16 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 552 bytes | 552.00 KiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
To https://huggingface.co/mesolitica/t5-small-finetuned-noisy-ms-en
   cd4e0aa..d0a4b4b  main -> main
